In [1]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

/home/ar600/anaconda2/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [3]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [4]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [42]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [43]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = zip(sample1, sample2)
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [11]:
tr = np.concatenate([np.ones(10), np.zeros(24)])
ts = np.concatenate([np.ones(4), np.zeros(12)])

In [12]:
print "p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind(tr, ts), alternative='greater')

p-value: 0.372930


In [15]:
banknotes = pd.read_csv('banknotes.txt', sep='\t')

In [16]:
banknotes.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [17]:
X = banknotes[['X1', 'X2', 'X3', 'X4', 'X5', 'X6']]
Y = banknotes.real

In [21]:
from sklearn import cross_validation, linear_model

train_data, test_data, train_labels, test_labels = cross_validation.train_test_split(X, Y, test_size = 0.25, 
                                                                                     random_state=1)

In [22]:
log_reg1 = linear_model.LogisticRegression()
log_reg2 = linear_model.LogisticRegression()

In [23]:
log_reg1.fit(train_data[['X1', 'X2', 'X3']], train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [26]:
log_reg2.fit(train_data[['X4', 'X5', 'X6']], train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [27]:
pred1 = log_reg1.predict(test_data[['X1', 'X2', 'X3']])
pred2 = log_reg2.predict(test_data[['X4', 'X5', 'X6']])

In [37]:
errors1 = np.abs(test_labels-pred1)
errors2 = np.abs(test_labels-pred2)

In [38]:
from sklearn import metrics
print metrics.classification_report(test_labels, pred1)

             precision    recall  f1-score   support

          0       0.74      0.87      0.80        23
          1       0.87      0.74      0.80        27

avg / total       0.81      0.80      0.80        50



In [39]:
print metrics.classification_report(test_labels, pred2)

             precision    recall  f1-score   support

          0       1.00      0.96      0.98        23
          1       0.96      1.00      0.98        27

avg / total       0.98      0.98      0.98        50



In [40]:
print np.mean(errors1)
print np.mean(errors2)

0.2
0.02


In [44]:
print "p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_rel(errors1, errors2))

p-value: 0.003297


In [45]:
print "95%% confidence interval for a difference between proportions: [%f, %f]" \
      % proportions_diff_confint_rel(errors1, errors2)

95% confidence interval for a difference between proportions: [0.059945, 0.300055]


In [50]:
mX = 541.5
m = 525
sigma = 100 
n = 100

Z = (mX-m)/(sigma/np.sqrt(n))

In [51]:
p = 1- stats.norm.cdf(Z)
print p

0.0494714680336


In [53]:
m0 = 200
weeks = np.array([49,58,75,110,112,132,151,276,281,362])
stats.wilcoxon(weeks - m0)

WilcoxonResult(statistic=17.0, pvalue=0.28450269791120752)

In [54]:
tr_cut = np.array([17,18,18,15,12,4,14,15,10])
tr_uncut = np.array([22,22,15,13,19,19,18,20,21,13,13,15])
stats.mannwhitneyu(tr_cut,tr_uncut, alternative='less')

MannwhitneyuResult(statistic=27.0, pvalue=0.029004992720873729)

In [55]:
challenger = pd.read_csv('challenger.txt', delimiter='\t')
challenger

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0
5,Jun18.83,22.2,0
6,Aug30.83,22.8,0
7,Nov28.83,21.1,0
8,Feb03.84,13.9,1
9,Apr06.84,17.2,1


In [56]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [68]:
inc = np.array(challenger[challenger.Incident==1].Temperature.values)
noinc = np.array(challenger[challenger.Incident==0].Temperature.values)

In [65]:
np.random.seed(0)

inc_mean = np.array(map(np.mean, get_bootstrap_samples(inc, 1000)))
noinc_mean = np.array(map(np.mean, get_bootstrap_samples(noinc, 1000)))

print "95% confidence interval:",  stat_intervals(inc_mean-noinc_mean, 0.05)

 95% confidence interval: [-8.06457589 -1.45040179]


In [62]:
challenger[challenger.Incident==1].Temperature.values

array([ 21.1,  13.9,  17.2,  21.1,  11.7,  23.9,  14.4])

In [66]:
mean_scores = zip(map(np.mean, inc_mean), map(np.mean, noinc_mean))
delta_mean_scores = map(lambda x: x[0] - x[1], mean_scores)

In [67]:
print "95% confidence interval ",  stat_intervals(delta_mean_scores, 0.05)

95% confidence interval  [-8.06457589 -1.45040179]


In [69]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

In [70]:
def get_random_combinations(n1, n2, max_combinations):
    index = range(n1 + n2)
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

In [71]:
def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [72]:
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [73]:
np.random.seed(0)
permutation_test(inc, noinc, max_permutations = 10000)

0.0057